# Calculation on facets #

In [3]:
import numpy as np

In [4]:
candidate_plane_1 = (1, 1, 1) # Obtained from x-ray diffraction

According Eq. 5, the facet corresponding to 
the crystallographic plane $(h_3, k_3, l_3)$, must satisfy 
$h_3+k_3+l_3 = 0$. We consider by brute force all possible planes 
$(h_3, k_3, l_3)$ subject to the condition $h_3+k_3+l_3 = 0$ for Miller indices 
greater than $-3$ and less than $3$.

In [21]:
def miller_idx_is_even(miller_idx):
    result = ((miller_idx % 2) == 0)
    return result

def miller_idx_is_odd(miller_idx):
    result = ((miller_idx % 2) == 1)
    return result

def all_miller_indices_are_even(plane):
    h, k, l = plane
    if (miller_idx_is_even(h)
        and miller_idx_is_even(k)
        and miller_idx_is_even(l)):
        result = True
    else:
        result = False
    return result

def all_miller_indices_are_odd(plane):
    h, k, l = plane
    if (miller_idx_is_odd(h)
        and miller_idx_is_odd(k)
        and miller_idx_is_odd(l)):
        result = True
    else:
        result = False
    return result

def plane_is_a_fcc_plane(plane):
    h, k, l = plane
    if (h == 0) and (k == 0) and (l == 0):
        result = False
    else:
        if (all_miller_indices_are_even(plane)
            or all_miller_indices_are_odd(plane)):
            result = True
        else:
            result = False
        return result
    
def plane_is_a_candidate_for_plane_3(plane):
    if plane_is_a_fcc_plane(plane) and (sum(plane) == 0):
        result = True
    else:
        result = False
    return result

In [30]:
max_miller_idx = 2

min_miller_idx = -max_miller_idx
candidate_subset_for_plane_3 = []

for h in range(min_miller_idx, max_miller_idx+1):
    for k in range(min_miller_idx, max_miller_idx+1):
        for l in range(min_miller_idx, max_miller_idx+1):
            plane = (h, k, l)
            if plane_is_a_candidate_for_plane_3(plane):
                candidate_subset_for_plane_3.append(plane)

print(candidate_subset_for_plane_3)

[(-2, 0, 2), (-2, 2, 0), (0, -2, 2), (0, 2, -2), (2, -2, 0), (2, 0, -2)]


The facet that corresponds to the crystallographic plane $(h_2, k_2, l_2)$, cannot be parallel to either
plane $(h_1, k_1, l_1)$ or $(h_3, k_3, l_3)$. We consider by brute force all possible planes to fullfill this 
condition for Miller indices greater than $-3$ and less than $3$.

In [23]:
def plane_is_not_a_candidate_for_plane_1(plane):
    h, k, l = plane
    if h == k == l:
        result = False
    else:
        result = True
    return result

def plane_is_not_a_candidate_for_plane_3(plane):
    result = (not plane_is_a_candidate_for_plane_3(plane))
    return result

def plane_is_a_candidate_for_plane_2(plane):
    if (plane_is_not_a_candidate_for_plane_1(plane)
        and plane_is_not_a_candidate_for_plane_3(plane) 
        and plane_is_a_fcc_plane(plane)):
        result = True  # Therefore plane 2 is not parallel to plane 1 nor 3.
    else:
        result = False
    return result

In [24]:
candidate_subset_for_plane_2 = []

for h in range(min_miller_idx, max_miller_idx+1):
    for k in range(min_miller_idx, max_miller_idx+1):
        for l in range(min_miller_idx, max_miller_idx+1):
            plane = (h, k, l)
            if plane_is_a_candidate_for_plane_2(plane):
                candidate_subset_for_plane_2.append(plane)

candidate_subset_for_plane_2

[(-2, -2, 0),
 (-2, -2, 2),
 (-2, 0, -2),
 (-2, 0, 0),
 (-2, 2, -2),
 (-2, 2, 2),
 (-1, -1, 1),
 (-1, 1, -1),
 (-1, 1, 1),
 (0, -2, -2),
 (0, -2, 0),
 (0, 0, -2),
 (0, 0, 2),
 (0, 2, 0),
 (0, 2, 2),
 (1, -1, -1),
 (1, -1, 1),
 (1, 1, -1),
 (2, -2, -2),
 (2, -2, 2),
 (2, 0, 0),
 (2, 0, 2),
 (2, 2, -2),
 (2, 2, 0)]

Here we calculate the sum of cosines of interfacial angles for all candidates of
$(h_2, k_2, l_2)$ and $(h_3, k_3, l_3)$ and check whether the sum is equal to $1$ (see Eq. 7).

In [32]:
def square_of_cosine_of_interfacial_angle(plane_A, plane_B):
    result = (np.dot(plane_A, plane_B)**2/ np.dot(plane_A, plane_A)
              / np.dot(plane_B, plane_B))
    return result

def sum_of_cosines_of_interfacial_angles(plane_1, plane_2, plane_3):
    result = (square_of_cosine_of_interfacial_angle(plane_1, plane_2)
              + square_of_cosine_of_interfacial_angle(plane_2, plane_3))
    return result

def sum_of_cosines_of_interfacial_angles_is_equal_to_unity(plane_1, 
                                                           plane_2, 
                                                           plane_3):
    tolerance = 1.0e-10 # Comparing floats

    if abs(1-sum_of_cosines_of_interfacial_angles(plane_1, plane_2, plane_3)) < tolerance:
        result = True 
    else:
        result = False
    return result

In [33]:
num_cases_where_sum_of_cosines_equals_unity = 0

for candidate_plane_2 in candidate_subset_for_plane_2:
    for candidate_plane_3 in candidate_subset_for_plane_3:
        tolerance = 1.0e-10
      
        if sum_of_cosines_of_interfacial_angles_is_equal_to_unity(candidate_plane_1,
                                                                  candidate_plane_2,
                                                                  candidate_plane_3):
            num_cases_where_sum_of_cosines_equals_unity += 1
            unformatted_msg = ("The sum of cosines of interfacial angles is "
                               "equal to 1 for:\n"
                               "    plane_1: ({}, {}, {})\n"
                               "    plane_2: ({}, {}, {})\n"
                               "    plane_3: ({}, {}, {})\n\n\n")
            msg = unformatted_msg.format(*candidate_plane_1, 
                                         *candidate_plane_2, 
                                         *candidate_plane_3)
            print(msg)

unformatted_msg = ("The number of cases considered where the sum of cosines of "
                   "interfacial angles is equal to 1: {}")
msg = unformatted_msg.format(num_cases_where_sum_of_cosines_equals_unity)
print(msg)

The number of cases considered where the sum of cosines of interfacial angles is equal to 1: 0
